Fase 1: EXTRACCION DE DATOS DE PELICULAS
Nuestro primer objetivo es extraer los datos de películas de la API. En Adalab nos hemos creado un API
muy sencilla que contiene la siguiente información:
Datos a extraer:
Título
Año de lanzamiento
Duración (en minutos)
Género
Contenido para adultos (sí o no)
El objetivo es extraer 100 películas de esta API utilizando el siguiente endpoint:
https://beta.adalab.es/resources/apis/pelis/pelis.json

In [1]:
## Importar las librerias necesarias
import requests ## para conectar con Api y descargar los datos
import pandas as pd ## para convertir los datos en formato tabla
import mysql.connector ## para conectar Paython con MySQL
import numpy as np ## para convertir formato nan de Paython a none de MySQL
from mysql.connector import Error ## captura errores de MySQL

In [2]:
## Función para llamar a Api y comprobar errores

def llamar_api(url):
    datos = requests.get(url)       ## llamamos a la APi
    print(f"Llamada al API, con respuesta : {datos.status_code}") ## muestra si la respuesta es correcta

    if datos.status_code != 200:
        print(f"Llamada falló, por el siguiente motivo: {datos.reason}")## si hay error nos indica el motivo
    else:
        return datos.json() ## nos devuelve los datos en formato json

In [3]:
## Extraer datos de la Api

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
datos_pelis = llamar_api(url)

Llamada al API, con respuesta : 200


In [4]:
## Muestra de los datos extraidos
datos_pelis
type(datos_pelis)

list

In [5]:
## Datos_pelis es una lista de diccionarios;eliminar id y subtitulos.
for peli in datos_pelis:
    if "id" in peli:
       del peli ["id"]
    if "subtitulos" in peli:
        del peli["subtitulos"]
print(datos_pelis)

[{'titulo': 'The Godfather', 'año': 1972, 'duracion': 175, 'genero': 'Crimen', 'adultos': False}, {'titulo': 'The Godfather Part II', 'año': 1974, 'duracion': 202, 'genero': 'Crimen', 'adultos': False}, {'titulo': 'Pulp Fiction', 'año': 1994, 'duracion': 154, 'genero': 'Crimen', 'adultos': True}, {'titulo': 'Forrest Gump', 'año': 1994, 'duracion': 142, 'genero': 'Drama', 'adultos': False}, {'titulo': 'The Dark Knight', 'año': 2008, 'duracion': 152, 'genero': 'Acción', 'adultos': False}, {'titulo': 'Fight Club', 'año': 1999, 'duracion': 139, 'genero': 'Drama', 'adultos': True}, {'titulo': 'Inception', 'año': 2010, 'duracion': 148, 'genero': 'Ciencia ficción', 'adultos': False}, {'titulo': 'The Matrix', 'año': 1999, 'duracion': 136, 'genero': 'Ciencia ficción', 'adultos': False}, {'titulo': 'The Shawshank Redemption', 'año': 1994, 'duracion': 142, 'genero': 'Drama', 'adultos': False}, {'titulo': 'Interstellar', 'año': 2014, 'duracion': 169, 'genero': 'Ciencia ficción', 'adultos': False},

In [6]:
## Convertir datos_pelis a tabla a través de pandas y mostrar los resultados
tabla_datos_pelis = pd.DataFrame(datos_pelis)
tabla_datos_pelis

,titulo,año,duracion,genero,adultos
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False
...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False
96,Requiem for a Dream,2000,102,Drama,True
97,Memento,2000,113,Thriller,True
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False


FASE 2: CREACIÓN DE LA BASE DE DATOS


In [7]:
## Conexión de Paython a MySQL

try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='111729',
    )
    print('Conexión exitosa')
except Error as tipo_error:
    print('Error al conectar:', tipo_error)

Conexión exitosa


In [8]:
## Crear base de datos desde Paython
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS bd_peliculas_adalab "
    mycursor.execute(query)
    print("Query exitosa")
except :
    print("Error.")

Query exitosa


In [9]:
## Antes de crear la tabla en la base de datos comprobar como son los datos.
tabla_datos_pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   titulo    100 non-null    object
 1   año       100 non-null    int64 
 2   duracion  100 non-null    int64 
 3   genero    100 non-null    object
 4   adultos   100 non-null    bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 3.4+ KB


In [10]:
## Comprobar si tiene algun null, si es así se cambia el formato para insertar en MySQL
tabla_datos_pelis.isnull().sum()

titulo      0
año         0
duracion    0
genero      0
adultos     0
dtype: int64

In [11]:
## Cambiar el nombre de una columna de la tabla
tabla_datos_pelis_act = tabla_datos_pelis.rename(columns={"adultos": "contenido_para_adultos"})
tabla_datos_pelis_act

,titulo,año,duracion,genero,contenido_para_adultos
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False
...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False
96,Requiem for a Dream,2000,102,Drama,True
97,Memento,2000,113,Thriller,True
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False


In [12]:
## Crear tabla peliculas en la base de datos
mycursor.execute("USE bd_peliculas_adalab") # Indicar en que base de datos se va crear la tabla

query = '''CREATE TABLE peliculas (
    id_peli INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion INT,
    genero VARCHAR(100),
    contenido_para_adultos BOOLEAN
);'''

mycursor.execute(query)
print("Tabla creada exitosamente")

Tabla creada exitosamente


3. INSERTAR LOS DATOS EN LA BASE DE DATOS CREADA: bd_peliculas_adalab

In [13]:
## Insertar datos
# Query de inserción
query_insertar = """
INSERT INTO peliculas (titulo,año,duracion,genero,contenido_para_adultos) 
VALUES (%s, %s, %s, %s, %s)
"""

datos = tabla_datos_pelis_act.values.tolist()## convertir tabla dataframe en lista
## al comprobar anteriormente que no hay valores nulos no se realiza el cambio de nan a none.

# Ejecutar inserción
mycursor.executemany(query_insertar, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")## cursor.rowcount cuenta cuantas entradas hay


100 registros insertados


FASE 4: OBTENER INFORMACION A PARTIR DE LOS DATOS
Una vez que tenemos toda la información, vamos a responder las siguientes preguntas utilizando consultas
en SQL

In [14]:
##¿Cuántas películas tienen una duración superior a 120 minutos?
#Incio del  cursor
mycursor = cnx.cursor()
# Indicar base de datos donde voy a realizar la consulta
mycursor.execute("USE bd_peliculas_adalab;")

query1 = "SELECT COUNT(id_peli) FROM peliculas WHERE duracion > 120;"

mycursor.execute(query1)
resultado = mycursor.fetchone() # uso de fetchone porque me muestra solo un resultado
print(resultado[0],"peliculas tienen una duración superior a 120 minutos")# [0] para que muestre solo el valor


59 peliculas tienen una duración superior a 120 minutos


In [15]:
##¿Cuántas películas tienen contenido adulto?

query2 = "SELECT COUNT(*) AS total_adultos FROM peliculas WHERE contenido_para_adultos = 1;"

mycursor.execute(query2)
resultado2 = mycursor.fetchone()
print(resultado2[0],"peliculas son de contenido para adultos")

47 peliculas son de contenido para adultos


In [16]:
#¿Cuál es la película más antigua registrada en la base de datos?
query3 = "SELECT titulo, año FROM peliculas ORDER BY año ASC LIMIT 1;"

mycursor.execute(query3)
resultado3 = mycursor.fetchall()
df = pd.DataFrame(resultado3,columns=["titulo","año"])# convertir resultado en un dataframe para visualizar con mas claridad.
df # mostrar dataframe

,titulo,año
0,Citizen Kane,1941


In [17]:
#Muestra el promedio de duración de las películas agrupado por género.
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="111729",
  database="bd_peliculas_adalab"
) # conexion a la base de datos

## Mostrar resultados directamente desde pandas con el metodo pd.read_sql_query
query4 = "SELECT AVG(duracion),genero FROM peliculas GROUP BY genero;"

df_promedio = pd.read_sql_query(query4,cnx)
df_promedio

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20496\466502599.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_promedio = pd.read_sql_query(query4,cnx)


,AVG(duracion),genero
0,154.2857,Crimen
1,126.2593,Drama
2,139.4444,Acción
3,136.3077,Ciencia ficción
4,139.6667,Romance
5,161.0000,Bélico
6,121.6667,Thriller
7,128.0000,Musical
8,159.8000,Fantasía
9,133.0000,Aventura


In [18]:
#¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

query5 ="SELECT año, COUNT(*) AS cantidad_peliculas FROM peliculas GROUP BY año ORDER BY cantidad_peliculas DESC"
df_pelis_año = pd.read_sql_query(query5,cnx)
df_pelis_año

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20496\478855863.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pelis_año = pd.read_sql_query(query5,cnx)


,año,cantidad_peliculas
0,2001,5
1,2013,4
2,1994,4
3,2008,4
4,1999,4
5,2017,4
6,2010,3
7,1998,3
8,2014,3
9,2000,3


In [19]:
#¿Cuál es el año con más películas en la base de datos?

df_pelis_año.head(1) # uso del metodo head() para mostrar la primera fila de la tabla anterior
            

,año,cantidad_peliculas
0,2001,5


In [20]:
#Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

query6 ="SELECT genero, COUNT(*) AS cantidad_peliculas FROM peliculas GROUP BY genero ORDER BY cantidad_peliculas DESC;"

df_generos = pd.read_sql_query(query6,cnx)
df_generos

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20496\1537469128.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_generos = pd.read_sql_query(query6,cnx)


,genero,cantidad_peliculas
0,Drama,27
1,Ciencia ficción,13
2,Acción,9
3,Animación,9
4,Crimen,7
5,Terror,7
6,Thriller,6
7,Fantasía,5
8,Romance,3
9,Aventura,3


In [21]:
#Muestra todas las películas cuyo título contenga la palabra "star" (puedes usar cualquier palabra)

query7 ="SELECT titulo FROM peliculas WHERE titulo LIKE '%star%';"

df_titulo_star = pd.read_sql_query(query7,cnx)
df_titulo_star

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20496\1904278944.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_titulo_star = pd.read_sql_query(query7,cnx)


,titulo
0,Star Wars: A New Hope
1,Star Wars: The Empire Strikes Back
2,Star Wars: Return of the Jedi


In [24]:
## Cerrar la conexion con MySQL

cnx.close()